# About 
This scripts removes as much shared peptides as possible with trivial methods.

Algorithm:
1. Read in the database
2. Loop through lines of database
3. Protein <- line starting with ">"
4. sequence <- other lines.
5. Split sequence at "K" and "R".
6. Create a list with the split sequences.
7. Create an equally sized list with the protein name for each split sequence. 
8. Create a DataFrame with all the proteins and splitted sequences.
9. If we want to replace I/L with L then set "modify_IL" to True.
10. Create a column with mapped decoys.
11. Create a column with mapped species.
12. Create a column with mapped "sequence length".
13. Keep only rows with "sequence length" > 7.
14. Drop duplicates.
15. Create a dataframe for number of protein per sequence by using - Groupby sequence (These are the splitted sequences still).
16. Count proteins for each sequence. 
17. Filter count dataframe to contain only sequences with count == 1.
18. Take only unique protein.
19. Make a list of the unique proteins. 
20. This protein contains the final result that we keep.
21. Read in the orignal db again and keep only proteins that exist in list created in 19).


In [20]:

import os 
import re
import pandas as pd
import numpy as np

In [21]:
# read in .fasta file and count shared peptides

os.chdir("/home/ptruong/git/dia_sum")

#human_fasta = "2021-06-15-decoys-reviewed-contam-UP000000625.fas"
#yeast_fasta = "2021-06-15-decoys-reviewed-contam-UP000002311"
#ecoli_fasta = "2021-06-15-decoys-reviewed-contam-UP000005640"

#filename = "database/2021-06-15/" + human_fasta
#filename = "database/napedro_3mixed_human_yeast_ecoli_20140403_iRT_reverse.fasta"
#filename = "database/2021-06-07/UP00000625_UP000002311_UP000005640.fasta"
#filename = "database/2021-09-15_malaria_yeast_PXD017705/UP000002311_UP000001450.fasta"
filename = "database/2022-05-25_pxd028185/SupplementaryFiles files/libraries and fastas/uniprot_human_25apr2019_yeastENO1.fasta"
# filename = "/home/ptruong/git/dia_sum/database/2021-11-02_PXD026600_ecoli/UP000000625_reviewed_ups1-ups2-sequences_2021-11-02.fasta"

file = open(filename, "r")


protein_list = []
sequence_list = []
for line in file: 
    if line[0] == ">":
        protein = line 
    else:
        sequence = line.rstrip()
        split_sequence = re.split(r"(?<=[KR])", sequence)
        split_sequence = list(dict.fromkeys(split_sequence))
        sequence_list += split_sequence
        protein_list += [protein for i in range(len(split_sequence))]
        

df_ = pd.DataFrame(np.array([protein_list, sequence_list]).T, columns = ["protein", "sequence"])
df = pd.DataFrame(np.array([protein_list, sequence_list]).T, columns = ["protein", "sequence"])


# If True replace I and L with common symbol

Amino acis I (Iso-leucine) and L (leucine) have the same weight and therefore the same peak in a mass-spec. We want to make them interchangable to see if it affects the final database.

In [3]:

mofidy_IL = True #Set the variable
if mofidy_IL == True:
    modify_IL_function = lambda x: x.replace("I", "L")
    df["sequence"] = df.sequence.map(modify_IL_function)
    print("All I and L are now L")
else:
    print("No I/L modification")

All I and L are now L


In [3]:
def decoy_map(protein):
    if protein.split("_")[0] == ">reverse":
        return True
    else:
        return False
    
    
df["decoy"] = df.protein.map(decoy_map)

df = df[df.decoy == False]


In [25]:
df

,protein,sequence,decoy
0,>sp|A0A385XJ53|INSA9_ECOLI Insertion element I...,MASVSLSCPSCSATDGVVR,False
1,>sp|A0A385XJ53|INSA9_ECOLI Insertion element I...,NGK,False
2,>sp|A0A385XJ53|INSA9_ECOLI Insertion element I...,STAGHQR,False
3,>sp|A0A385XJ53|INSA9_ECOLI Insertion element I...,YLCSHCR,False
4,>sp|A0A385XJ53|INSA9_ECOLI Insertion element I...,K,False
...,...,...,...
130356,>P01579ups|IFNG_HUMAN_UPS Interferon Gamma (Ch...,ALHELLQVMAELSPAAK,False
130357,>P01579ups|IFNG_HUMAN_UPS Interferon Gamma (Ch...,TGK,False
130358,>P01579ups|IFNG_HUMAN_UPS Interferon Gamma (Ch...,SQMLFR,False
130359,>P01579ups|IFNG_HUMAN_UPS Interferon Gamma (Ch...,GR,False


In [4]:
specie_map = lambda x: x.split("_")[1].split(" ")[0]
df["specie"] = df.protein.map(specie_map)

In [27]:
n_protein = len(df.protein.unique())
n_protein_ecoli  = len(df[df["specie"] == "ECOLI"].protein.unique())
n_protein_human = len(df[df["specie"] == "HUMAN"].protein.unique())
n_protein_yeast = len(df[df["specie"] == "YEAST"].protein.unique())

In [28]:
df.specie.unique()

array(['ECOLI', 'HUMAN'], dtype=object)

In [29]:
print("Unfiltered protein statistics:")
print(f"All proteins: {n_protein}")
print(f"ECOLI proteins: {n_protein_ecoli}")
print(f"HUMAN proteins: {n_protein_human}")
print(f"YEAST protein: {n_protein_yeast}")

Unfiltered protein statistics:
All proteins: 4438
ECOLI proteins: 4390
HUMAN proteins: 48
YEAST protein: 0


In [30]:
df["seq_length"] = df.sequence.str.len()
df = df[df["seq_length"] > 7]
df.drop("seq_length", axis = 1, inplace = True)
df.drop_duplicates(inplace=True)


/home/ptruong/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ptruong/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


# Find protein with only one sequence

In [31]:
counted_df = df.groupby("sequence").count().sort_values(by = "protein", ascending = False)
counted_df

,protein,decoy,specie
sequence,,,
PYPLETMLR,12,12,12
LSLDSALPDR,12,12,12
LHCMQHWYNLSDGAMEDALYELASMR,12,12,12
TALNLEYMK,12,12,12
NDNQLAMLFTLANLFR,12,12,12
...,...,...,...
GVPGALSSATEALK,1,1,1
GVPLLTVLFMSSVMLPLFMAEGTSLDK,1,1,1
GVPLNSLMLSGALTSLVVLLNYLLPQK,1,1,1


In [32]:
single_protein_seq = counted_df[counted_df.protein == 1]

# investigate if a single protein sequence has proteins containing single sequences.

In [33]:
df_single_seq = df[df.sequence.isin(single_protein_seq.index)]

In [34]:
df_single_seq.groupby("sequence").count().max()

protein    1
decoy      1
specie     1
dtype: int64

In [35]:
# seems fine, keep these proteins 
len(df_single_seq.protein.unique())

4350

In [36]:
len(df.protein.unique())

4421

In [37]:
keep_list = df_single_seq.protein.unique()

In [38]:
keep_list

array(['>sp|A0A385XJK5|YPAB_ECOLI Protein YpaB OS=Escherichia coli (strain K12) OX=83333 GN=ypaB PE=4 SV=1\n',
       '>sp|A0A385XJL2|YGDT_ECOLI Protein YgdT OS=Escherichia coli (strain K12) OX=83333 GN=ygdT PE=4 SV=1\n',
       '>sp|A5A605|YKFM_ECOLI Uncharacterized protein YkfM OS=Escherichia coli (strain K12) OX=83333 GN=ykfM PE=4 SV=1\n',
       ...,
       '>P63279ups|UBC9_HUMAN_UPS SUMO-conjugating enzyme UBC9 (Chain 1-158) - Homo sapiens (Human)\n',
       '>O76070ups|SYUG_HUMAN_UPS Gamma-synuclein (Chain 1-127) - Homo sapiens (Human)\n',
       '>P01579ups|IFNG_HUMAN_UPS Interferon Gamma (Chain 23-166) - Homo sapiens (Human)\n'],
      dtype=object)

# Read in both fasta files and compare results.

This is the function to check if the protein is in "keep_list", if it is in keep it from the original.

In [39]:

os.chdir("/home/ptruong/git/dia_sum")
#filename = "database/2021-06-07/UP00000625_UP000002311_UP000005640.fasta"
#filename = "database/2022-05-25_pxd028185/SupplementaryFiles files/libraries and fastas/uniprot_human_25apr2019_yeastENO1.fasta"
#filename = "database/napedro_3mixed_human_yeast_ecoli_20140403_iRT_reverse.fasta"

filename = "/home/ptruong/git/dia_sum/database/2021-11-02_PXD026600_ecoli/UP000000625_reviewed_ups1-ups2-sequences_2021-11-02.fasta"

file = open(filename, "r")

#file_w = open("database/2021-09-13_IL_aminoacids/UP00000625_UP000002311_UP000005640_no_shared_peptides_IL_modified.fasta", "w")

file_w = open("/home/ptruong/git/dia_sum/database/2021-11-02_PXD026600_ecoli/UP000000625_reviewed_ups1-ups2-sequences_2021-11-02_noShared_mod_IL.fasta", "w")



for line in file:
    if line[0] == ">":
        protein = line 
        file_w.write(protein)
    else:
        if protein in keep_list:
            sequence = line
            file_w.write(sequence)
        

# Protein database analysis

Here we analyse the amount of proteins in each library

In [5]:

def read_in_fasta(filename):
    file = open(filename, "r")

    protein_list = []
    sequence_list = []
    for line in file: 
        if line[0] == ">":
            protein = line 
        else:
            sequence = line.rstrip()
            #sequence_list += split_sequence

            #protein_list += [protein for i in range(len(sequence))]
            sequence_list.append(sequence)
            protein_list.append(protein)

    df = pd.DataFrame(np.array([protein_list, sequence_list]).T, columns = ["protein", "sequence"])
    df["decoy"] = df.protein.map(decoy_map)
    df["specie"] = df.protein.map(specie_map)
    return df


In [10]:
# filename = "database/2021-06-07/UP00000625_UP000002311_UP000005640.fasta"
# filename_no_shared = "database/2021-09-13_IL_aminoacids/UP00000625_UP000002311_UP000005640_no_shared_peptides.fasta"
# filename_no_shared_IL_mod = "database/2021-09-13_IL_aminoacids/UP00000625_UP000002311_UP000005640_no_shared_peptides_IL_modified.fasta"

filename = "/home/ptruong/git/dia_sum/database/2022-05-27-decoys-reviewed-contam-UP000005640.fas"

#filename = "database/2022-05-25_pxd028185/SupplementaryFiles files/libraries and fastas/uniprot_human_25apr2019_yeastENO1.fasta"
filename_no_shared = "database/2022-05-25_pxd028185/SupplementaryFiles files/libraries and fastas/uniprot_human_25apr2019_yeastENO1_noShared.fasta"
filename_no_shared_IL_mod = "database/2022-05-25_pxd028185/SupplementaryFiles files/libraries and fastas/uniprot_human_25apr2019_yeastENO1_noShared_IL_modified.fasta"


#filename = "/home/ptruong/git/dia_sum/database/2021-11-02_PXD026600_ecoli/UP000000625_reviewed_ups1-ups2-sequences_2021-11-02.fasta"
#filename_no_shared = "/home/ptruong/git/dia_sum/database/2021-11-02_PXD026600_ecoli/UP000000625_reviewed_ups1-ups2-sequences_2021-11-02_noShared.fasta"
#filename_no_shared_IL_mod = "/home/ptruong/git/dia_sum/database/2021-11-02_PXD026600_ecoli/UP000000625_reviewed_ups1-ups2-sequences_2021-11-02_noShared_mod_IL.fasta"

In [11]:
db = read_in_fasta(filename)
db_no_shared = read_in_fasta(filename_no_shared)
db_no_shared_IL_mod = read_in_fasta(filename_no_shared_IL_mod)

In [12]:
db

,protein,sequence,decoy,specie
0,>contam_sp|O00762|UBE2C_HUMAN Ubiquitin-conjug...,MASQNRDPAATSVAAARKGAEPSGGAARGPVGKRLQQELMTLMMSG...,False,sp|O00762|UBE2C
1,">contam_sp|O43790|KRT86_HUMAN Keratin, type II...",MTCGSYCGGRAFSCISACGPRPGRCCITAAPYRGISCYRGLTGGFG...,False,sp|O43790|KRT86
2,">contam_sp|O76009|KT33A_HUMAN Keratin, type I ...",MSYSCGLPSLSCRTSCSSRPCVPPSCHGCTLPGACNIPANVSNCNW...,False,sp|O76009|KT33A
3,">contam_sp|O76011|KRT34_HUMAN Keratin, type I ...",MLYAKPPPTINGIKGLQRKERLKPAHIHLQQLTCFSITCSSTMSYS...,False,sp|O76011|KRT34
4,">contam_sp|O76013|KRT36_HUMAN Keratin, type I ...",MATQTCTPTFSTGSIKGLCGTAGGISRVSSIRSVGSCRVPSLAGAA...,False,sp|O76013|KRT36
...,...,...,...,...
40835,">sp|U3KPV4|A3LT2_HUMAN Alpha-1,3-galactosyltra...",MALKEGLRAWKRIFWRQILLTLGLLGLFLYGLPKFRHLEALIPMGV...,False,HUMAN
40836,>sp|W5XKT8|SACA6_HUMAN Sperm acrosome membrane...,MALLALASAVPSALLALAVFRVPAWACLLCFTTYSERLRICQMFVG...,False,HUMAN
40837,>sp|W6CW81|PYDC5_HUMAN Pyrin domain-containing...,MESKYKEILLLTSLDNITDEELDRFKCFLPDEFNIATGKLHTLNST...,False,HUMAN
40838,>sp|X6R8D5|GUCNB_HUMAN Putative uncharacterize...,MGRKEHESPSQPHMCGWEDSQKPSVPSHGPKTPSCKGVKAPHSSRP...,False,HUMAN


In [44]:
db_no_shared

,protein,sequence,decoy,specie
0,>sp|A0A385XJK5|YPAB_ECOLI Protein YpaB OS=Esch...,MTLLQVHNFVDNSGRKKWLSRTLGQTRCPGKSMGREKFVKNNCSAIS,False,ECOLI
1,>sp|A0A385XJL2|YGDT_ECOLI Protein YgdT OS=Esch...,MLSTESWDNCEKPPLLFPFTALTCDETPVFSGSVLNLVAHSVDKYGIG,False,ECOLI
2,>sp|A5A605|YKFM_ECOLI Uncharacterized protein ...,MRLHVKLKEFLSMFFMAILFFPAFNASLFFTGVKPLYSIIKCSTEI...,False,ECOLI
3,>sp|A5A611|YMGI_ECOLI Uncharacterized protein ...,MSYSSFKIILIHVKNIIPIITATLILNYLNNSERSLVKQILIEDEI...,False,ECOLI
4,>sp|A5A612|YMGJ_ECOLI Uncharacterized protein ...,MSRRYSSATITLVILFCRRPHCRKISVRQYYQKRVIFYPTTLQRTA...,False,ECOLI
...,...,...,...,...
4345,>P01127ups|PDGFB_HUMAN_UPS Platelet-derived gr...,SLGSLTIAEPAMIAECKTRTEVFEISRRLIDRTNANFLVWPPCVEV...,False,HUMAN
4346,>P68871ups|HBB_HUMAN_UPS Hemoglobin subunit be...,VHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFG...,False,HUMAN
4347,>P63279ups|UBC9_HUMAN_UPS SUMO-conjugating enz...,MSGIALSRLAQERKAWRKDHPFGFVAVPTKNPDGTMNLMNWECAIP...,False,HUMAN
4348,>O76070ups|SYUG_HUMAN_UPS Gamma-synuclein (Cha...,MGSSHHHHHHSSGLVPRGSHMDVFKKGFSIAKEGVVGAVEKTKQGV...,False,HUMAN


In [45]:
db_no_shared_IL_mod

,protein,sequence,decoy,specie
0,>sp|A0A385XJK5|YPAB_ECOLI Protein YpaB OS=Esch...,MTLLQVHNFVDNSGRKKWLSRTLGQTRCPGKSMGREKFVKNNCSAIS,False,ECOLI
1,>sp|A0A385XJL2|YGDT_ECOLI Protein YgdT OS=Esch...,MLSTESWDNCEKPPLLFPFTALTCDETPVFSGSVLNLVAHSVDKYGIG,False,ECOLI
2,>sp|A5A605|YKFM_ECOLI Uncharacterized protein ...,MRLHVKLKEFLSMFFMAILFFPAFNASLFFTGVKPLYSIIKCSTEI...,False,ECOLI
3,>sp|A5A611|YMGI_ECOLI Uncharacterized protein ...,MSYSSFKIILIHVKNIIPIITATLILNYLNNSERSLVKQILIEDEI...,False,ECOLI
4,>sp|A5A612|YMGJ_ECOLI Uncharacterized protein ...,MSRRYSSATITLVILFCRRPHCRKISVRQYYQKRVIFYPTTLQRTA...,False,ECOLI
...,...,...,...,...
4337,>P63165ups|SUMO1_HUMAN_UPS Small ubiquitin-rel...,MSPILGYWKIKGLVQPTRLLLEYLEEKYEEHLYERDEGDKWRNKKF...,False,HUMAN
4338,>P51965ups|UB2E1_HUMAN_UPS Ubiquitin-conjugati...,HHHHHHMSDDDSRASTSSSSSSSSNQQTEKETNTPKKKESKVSMSK...,False,HUMAN
4339,>P12081ups|SYHC_HUMAN_UPS Histidyl-tRNA synthe...,MAERAALEELVKLQGERVRGLKQQKASAELIEEEVAKLLKLKAQLG...,False,HUMAN
4340,>P99999ups|CYC_HUMAN_UPS Cytochrome c (Chain 2...,GDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNLHGLFGRKTGQAPGY...,False,HUMAN


In [11]:

def print_db_proteins(db):
    n_protein = len(db.protein.unique())
    n_protein_ecoli = len(db[db["specie"] == "ECOLI"].protein.unique())
    n_protein_human = len(db[db["specie"] == "HUMAN"].protein.unique())
    n_protein_yeast = len(db[db["specie"] == "YEAST"].protein.unique())

    print(f"All proteins: {n_protein}")
    print(f"ECOLI proteins: {n_protein_ecoli}")
    print(f"HUMAN proteins: {n_protein_human}")
    print(f"YEAST protein: {n_protein_yeast}")



In [12]:
print("Unfiltered db:")
print_db_proteins(db)

Unfiltered db:
All proteins: 20416
ECOLI proteins: 0
HUMAN proteins: 20415
YEAST protein: 1


In [48]:
print("No shared:")
print_db_proteins(db_no_shared)

No shared:
All proteins: 4350
ECOLI proteins: 4302
HUMAN proteins: 48
YEAST protein: 0


In [49]:
print("No shared IL modified:")
print_db_proteins(db_no_shared_IL_mod)

No shared IL modified:
All proteins: 4342
ECOLI proteins: 4302
HUMAN proteins: 40
YEAST protein: 0


In [13]:
def get_db_proteins(db):
    n_protein = len(db.protein.unique())
    n_protein_ecoli = len(db[db["specie"] == "ECOLI"].protein.unique())
    n_protein_human = len(db[db["specie"] == "HUMAN"].protein.unique())
    n_protein_yeast = len(db[db["specie"] == "YEAST"].protein.unique())
    return n_protein, n_protein_ecoli, n_protein_human, n_protein_yeast


In [14]:
db_stats = pd.DataFrame([list(get_db_proteins(db)), list(get_db_proteins(db_no_shared)), list(get_db_proteins(db_no_shared_IL_mod))],
            index = ["Unfiltered", "No_shared", "No_shared_IL_mod"],
            columns = ["all", "ecoli", "human", "yeast"]).T

In [15]:
db_stats

,Unfiltered,No_shared,No_shared_IL_mod
all,40840,20113,20111
ecoli,0,0,0
human,20304,20112,20110
yeast,0,1,1


In [19]:
db_no_shared[db_no_shared.specie == "YEAST"].protein.unique()

array(['>sp|P00924|ENO1_YEAST Enolase 1 OS=Saccharomyces cerevisiae (strain ATCC 204508 / S288c) OX=559292 GN=ENO1 PE=1 SV=3\n'],
      dtype=object)